1) Напишем функцию, аргументом которой будет являться sql - запрос и результатом - pd DataFrame
2) Напишем функцию, которая будет определять числовые колонки в pd DataFrame и считать для них мин, макс и среднюю
3) "Упакуем" в один проект 

In [2]:
import pandas as pd
import numpy as np
import psycopg2

In [85]:
def query_db(
    query, 
    dbname = 'dvdrental', 
    user = 'postgres',
    host = 'localhost',
    port = '5432',
    password = '123'
    ):
    engine = psycopg2.connect(f"dbname={dbname} user={user} host={host} port={port} password={password}")
    df = pd.read_sql(query, con=engine)
    return df

def get_numeric_stats(df, columns_to_exclude = []):
    data_types = df.dtypes
    numeric_columns = data_types.loc[(data_types == int)|(data_types == float)].index.to_list()
    for col in columns_to_exclude:
        numeric_columns.remove(col)
    num_stats = df[numeric_columns].describe().loc[['min', 'mean', 'max']]
    return num_stats

In [28]:
# Создадим запрос к БД dvdrental
query = """
WITH unpaid_rentals_table AS 
(
	SELECT 
		rent.rental_id AS rental_id,
		rent.customer_id AS customer_id,
		pmt.amount AS payment
	FROM 
		rental AS rent
	LEFT JOIN 
		payment AS pmt
	ON 
		rent.rental_id = pmt.rental_id
	WHERE 
		pmt.amount IS NULL
),
bad_customers_table AS (
	SELECT 
		ur.customer_id,
		COUNT (DISTINCT ur.rental_id) AS unpaid_rentals
	FROM 
		unpaid_rentals_table AS ur
	GROUP BY 
		ur.customer_id
	ORDER BY
		unpaid_rentals DESC
)
SELECT 
	bc.customer_id AS customer_id,
	cust.first_name AS first_name,
	cust.last_name AS last_name,
	addr.phone AS phone_number,
	bc.unpaid_rentals AS unpaid_rentals
FROM 
	bad_customers_table AS bc
LEFT JOIN 
	customer AS cust
ON bc.customer_id = cust.customer_id
LEFT JOIN
	address AS addr
ON cust.address_id = addr.address_id;

"""
df = query_db(query = query)
df

C:\Users\D\AppData\Local\Temp\ipykernel_4740\1965639782.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=engine)


,customer_id,first_name,last_name,phone_number,unpaid_rentals
0,307,Joseph,Joy,573441801529,10
1,293,Mae,Fletcher,96604821070,10
2,316,Steven,Curley,139378397418,10
3,295,Daisy,Bates,816436065431,10
4,322,Jason,Morrissey,972574862516,9
...,...,...,...,...,...
525,203,Tara,Ryan,652218196731,1
526,234,Claudia,Fuller,630424482919,1
527,419,Chad,Carbone,987306329957,1
528,420,Jacob,Lance,546793516940,1


In [14]:
# Создадим запрос к БД demo_medium
query = 'select * from bookings.aircrafts'
dbname = 'demo'
df2 = query_db(query = query, dbname = dbname)
df2

C:\Users\D\AppData\Local\Temp\ipykernel_4740\1965639782.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=engine)


,aircraft_code,model,range
0,773,Boeing 777-300,11100
1,763,Boeing 767-300,7900
2,SU9,Sukhoi SuperJet-100,3000
3,320,Airbus A320-200,5700
4,321,Airbus A321-200,5600
5,319,Airbus A319-100,6700
6,733,Boeing 737-300,4200
7,CN1,Cessna 208 Caravan,1200
8,CR2,Bombardier CRJ-200,2700


In [92]:
print(get_numeric_stats(df))

      customer_id  unpaid_rentals
min      1.000000        1.000000
mean   301.790566        2.739623
max    599.000000       10.000000
